In [1]:
import json
import numpy as np
import pickle as pkl
import re
import pandas as pd
from datetime import datetime
from graph_tool import Graph, GraphView
from graph_tool.topology import label_largest_component
from tqdm import tqdm
from gt_utils import extract_nodes, extract_edges

In [2]:
path = 'data/digg/graph.csv'

In [3]:
df = pd.read_csv(path, header=None, names=['mutual', 'date', 'src', 'tar'])

In [4]:
all_nodes = (set(df['src'].unique()) | set(df['tar'].unique()))

In [23]:
print('the node ids are not consecutive')
len(all_nodes), max(all_nodes)

the node ids are not consecutive


(279631, 336225)

In [5]:
node_index = {n: i for i, n in enumerate(all_nodes)}

In [6]:
g = Graph(directed=False)
g.add_vertex(len(node_index))
edges = set()
for r in tqdm(df.as_matrix(), total=df.shape[0]):
    mutual, _, s, t = r
    s, t = node_index[s], node_index[t]
    edges.add(tuple(sorted((s, t))))

100%|██████████| 1731658/1731658 [00:14<00:00, 123024.59it/s]


In [7]:
g.add_edge_list(edges)

In [8]:
g.save('data/digg/graph.gt')

In [9]:
path = 'data/digg/votes.csv'
df = pd.read_csv(path, header=None, names=['date', 'voter', 'story'])

In [10]:
cascades = df.groupby('story')

In [11]:
cnts = cascades.count()

In [25]:
sorted_story = cnts.sort_values(by='voter', ascending=False)

In [26]:
sorted_story.head(5)

,date,voter
story,,
714,24099,24099
1123,8521,8521
502,8492,8492
2719,8057,8057
2050,7825,7825


In [36]:
def get_connected_infected_nodes(cascades, idx):
    c = cascades.get_group(idx)
    c['date'] -= c['date'].min()
    c['voter'] = c['voter'].map(lambda k: int(node_index.get(k, -1)))  # some voter is not in the graph, strange
    c = c[c['voter'] >= 0]  # filter them out    
    
    voter2time = dict(zip(c['voter'].as_matrix(), c['date'].as_matrix()))
    infected_nodes = c['voter'].as_matrix()
    
    # extract the most voted story subgraph
    # all nodes in this subgraph voted the story
    vfilt = g.new_vertex_property('bool')
    vfilt.a = False
    for i in infected_nodes:
        vfilt[i] = True
    gv = GraphView(g, vfilt=vfilt, directed=False)        
    
    # get the connected component
    cc_label = label_largest_component(gv)
    infected_nodes_connected = np.nonzero(cc_label.a)[0]
    
    # print('cascade size', sum(cc_label.a))
    
    infection_times = np.ones(g.num_vertices()) * -1
    
    for i in infected_nodes_connected:
        infection_times[i] = voter2time[i]    
        
    return infection_times

In [39]:
indices = sorted_story.index[:50]

In [41]:
infection_times_list = [get_connected_infected_nodes(cascades, idx) for idx in tqdm(indices)]

  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
100%|██████████| 50/50 [01:15<00:00,  1.36s/it]


In [42]:
list(map(lambda l: len((l>=0).nonzero()[0]), infection_times_list))

[8653,
 2660,
 1946,
 1624,
 1849,
 1748,
 1628,
 1844,
 1387,
 1787,
 2202,
 2627,
 2141,
 1407,
 1454,
 2071,
 1188,
 2030,
 1471,
 1083,
 1165,
 1109,
 1129,
 1268,
 1358,
 1393,
 1414,
 1381,
 1163,
 1209,
 1314,
 961,
 1780,
 1067,
 1772,
 1212,
 1819,
 1219,
 1416,
 1322,
 909,
 1084,
 873,
 1060,
 893,
 1179,
 1077,
 929,
 989,
 1004]

In [173]:
set((infection_times >= 0).nonzero()[0]) == set(infected_nodes_connected)

True

In [45]:
for i, infection_times in enumerate(infection_times_list[1:11]):
    with open('data/digg/cascade_{}.pkl'.format(i), 'wb') as f:
        pkl.dump(infection_times, f)